In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as style
import matplotlib.dates as mdates
import plotly.graph_objs as go
from plotly.subplots import make_subplots
'''
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from ta.volatility import BollingerBands
from datetime import datetime, timedelta
from scipy.stats import norm
'''
SP_500_database = pd.read_csv("database2.csv", index_col=0).round(2)

In [6]:
import yfinance as yf
msft=yf.Ticker('MSFT')
msft_data = msft.history(period='max')
msft_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13,0.06,0.06,0.06,0.06,1031788800,0.0,0.0
1986-03-14,0.06,0.07,0.06,0.06,308160000,0.0,0.0
1986-03-17,0.06,0.07,0.06,0.07,133171200,0.0,0.0
1986-03-18,0.07,0.07,0.06,0.06,67766400,0.0,0.0
1986-03-19,0.06,0.06,0.06,0.06,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2020-07-21,213.66,213.94,208.03,208.75,38105800,0.0,0.0
2020-07-22,209.20,212.30,208.39,211.75,49605700,0.0,0.0
2020-07-23,207.19,210.92,202.15,202.54,67457000,0.0,0.0


Everytime you edit any cell click run on that cell

# user inputs: 

In [9]:
ticker = "AAPL" # Uppercase stock ticker in the S&P 500

start_date = '01-01-2019' # Enter start date in format 'MM-DD-YYYY'
end_date = '01-01-2020' # Enter end date in format 'MM-DD-YYYY'

enable_RSI = False # True to filter based on RSI condition, False to not filter
RSI_period = 14 # parameter period for RSI
# RSI_conditional_value = 40  # To be considered a falling knife RSI has to be outside of this value (oversold value)
show_RSI_overbought_line = True # shows a straight line to visualize the overbought value (100 - RSI_conditional_value)
show_RSI_oversold_line = True # shows a straight line to visualize the oversold value (RSI_conditional_value)

enable_MACD = False # True to filter based on MACD condition, False to not filter
MACD_short_term_period = 12 # parameter for short term period for MACD
MACD_long_term_period = 26 # parameter for long term period for MACD
MACD_signal_period = 9 # parameter for signal period for MACD
MACD_conditional_value = -.01 # To be considered a falling knife MACD has to be outside of this value
show_MACD_signal_line = True # True to graph the MACD signal line

enable_RETURN = False # True to filter and graph based on return condition, False to not filter
RETURN_conditional_value = -.025 # To be considered a falling knife the percentage change of the stock has to be less than this value

enable_BOLLINGER_BANDS = True # True to filter and graph based on return condition, False to not filter
BOLLINGER_BANDS_period = 20 # number of days in smoothing period (usually 20)
BOLLINGER_BANDS_std = 2 # number of standard deviations (usually 2)



enable_STOCHASTICOSCILLATOR = False
STOCHASTICOSCILLATOR_period = 14
STOCHASTICOSCILLATOR_sma_period = 3
STOCHASTICOSCILLATOR_conditional_value = 20
show_STOCHASTICOSCILLATOR_signal_line = False



graph_RSI = False # True to graph, False to not
graph_closing_price = False # True to graph, False to not graph

# user should only click run on the cells below.  DO NOT EDIT CELLS BELOW UNLESS YOU KNOW WHAT YOU ARE EDITING

Run this next cell every time you edit some thing in the user input cell

In [10]:
# cleans aznd reformats the database based on the ticker
ticker_database = SP_500_database[SP_500_database["Ticker"]==ticker]
ticker_database['Date'] = pd.to_datetime(ticker_database['Date'])

#print(ticker_database.head(-5))
#filters database based on your choosen start and end date
#print(ticker_database.head(-5))



# creates indicator data
ticker_database['MACD'] = MACD(ticker_database["Adj Close"], n_slow=MACD_short_term_period, n_fast=MACD_long_term_period, n_sign=MACD_signal_period).macd()
ticker_database['MACD Signal'] = MACD(ticker_database["Adj Close"], n_slow=MACD_short_term_period, n_fast=MACD_long_term_period, n_sign=MACD_signal_period).macd_signal()


ticker_database['RSI'] = RSIIndicator(ticker_database["Adj Close"], RSI_period).rsi()

ticker_database['Return'] = ticker_database["Adj Close"].pct_change()


ticker_database['Stochastic Oscillator'] = StochasticOscillator(high=ticker_database['High'], low=ticker_database['Low'], close=ticker_database['Adj Close'], n=STOCHASTICOSCILLATOR_period, d_n=STOCHASTICOSCILLATOR_sma_period).stoch()

BB = BollingerBands(ticker_database['Adj Close'], BOLLINGER_BANDS_period, BOLLINGER_BANDS_std)
ticker_database['Bollinger Bands High'] = BB.bollinger_hband()
ticker_database['Bollinger Bands Low'] = BB.bollinger_lband()


ticker_database = ticker_database.loc[start_date:end_date]

ticker_database['RSI Conditional Value'] = 100 - norm.ppf(.95, np.mean(ticker_database['RSI']), np.std(ticker_database['RSI']))
print(RSI_conditional_value)


ticker_database_filtered = ticker_database

# these are the condition statements below: I couldnt figure out how to keep them in one line so..
print("\nfiltering data based on conditions...")

if enable_RSI:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered['RSI'] < RSI_conditional_value]

if enable_MACD:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered["MACD"] <= MACD_conditional_value]

if enable_RETURN:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered["Return"] <= RETURN_conditional_value]

if enable_STOCHASTICOSCILLATOR:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered['Stochastic Oscillator']  <= STOCHASTICOSCILLATOR_conditional_value]




# fallingKniveDf = df[df['Adj Close'] < df['SMA']]

print("There are " + str(len(ticker_database_filtered)) + " falling knives")
print(ticker_database_filtered)


KeyError: 'Ticker'

This next cell manages the graphing

In [ ]:
# Create figure
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, row_heights=[.15, .15, .15, .55])

#1. vwap
'''
fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['Bollinger Bands High'],
                    name='Bollinger Bands High'),
                    row=1, col=1)

fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['Bollinger Bands Low'],
                    name='Bollinger Bands High'),
                    row=1, col=1)

fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['Bollinger Bands Low'],
                    name='Bollinger Bands High'),
                    row=1, col=1)

'''
fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['RSI'],
                    name='RSI'),
                    row=2, col=1)

fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['RSI'],
                    mode='markers',
                    name='fn RSI'),
                    row=2, col=1)
if show_RSI_overbought_line:
    fig.add_trace(go.Scatter(x=[ticker_database.index.min(), ticker_database.index.max()], y=[100 - RSI_conditional_value, 100 - RSI_conditional_value],
                    name='RSI overbought'),
                    row=2, col=1)
if show_RSI_oversold_line:
    fig.add_trace(go.Scatter(x=[ticker_database.index.min(), ticker_database.index.max()], y=[RSI_conditional_value, RSI_conditional_value],
                    name='RSI oversold'),
                    row=2, col=1)


fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['MACD'], 
                    name='MACD'), 
                    row=3, col=1)
if show_MACD_signal_line:
    fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['MACD Signal'], 
                    name='MACD Signal'), 
                    row=3, col=1)


fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['MACD'],
                    mode='markers',
                    name='fn MACD'),
                    row=3, col=1)

fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['Adj Close'],
                    name='Closing Price'), 
                    row=4, col=1)

fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['Adj Close'],
                    mode='markers',
                    name='fn Closing Price'),
                    row=4, col=1)



# Add range slider
fig.update_layout(
    showlegend=False,
    title_text="Finding Falling Knives",
    #yaxis_title="Stochastic Oscillator",
    yaxis2_title="RSI",
    yaxis3_title="MACD",
    yaxis4_title="Closing Price",
    xaxis4_title="Timeframe",
    xaxis4_rangeslider_visible=True,
    height=800,
    width=1000,
    xaxis4=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    )
)

fig.show(config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})

MA
Bolling bands*
fibonici sequence better
elliot wave
price to volume ratio
oob
adi
*vwap: where the price starts to accumate, ex: does 1 person by 100,000 shares or does (P is above or below vwap)
put and call ratio

letting the user graph
lines, circles



9 21 45 90
normal distubition
3 version, extreme, moderate, light
